In [ ]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [ ]:
# SEE SOME OPTIONAL INFORMATION ABOUT THE DATAFRAME

# print(df.head())
# print(df.describe())
print(df)
print(df.Variety.unique(), "\nthere are ", len(df.Variety.unique()), " unique varieties")

In [ ]:
# OPTIONAL CORRELATION CHECKER

#these were selected mainly due to their dtype
potential_feature_columns = ['Location', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class', 'CEC', 'pH', 'Clay', 'Silt', 'Sand', 'Area']
# print(potential_feature_columns)
potential_output_column = df.Yield

potential_X = df.loc[:, potential_feature_columns]
potential_y = potential_output_column

for x in potential_X:
    print("%s has correlation %s with yield" % (x, np.correlate(df[x], potential_y)))

In [ ]:
# CURRENTLY UNUSED OPTIONAL ONEHOT ENCODER

from sklearn.preprocessing import LabelEncoder
from pandas import Series
le = LabelEncoder()
integer_encoded = le.fit_transform(df.Variety)
# le.classes_


from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(onehot_encoded)

# df['VarietyEncode'] = onehot_encoded
# print(pd.get_dummies(df.Variety))
# print(df.Variety)
    
from numpy import argmax
inverted = le.inverse_transform([argmax(onehot_encoded[300, :])])
print([argmax(onehot_encoded[300, :])])
verted = le.transform(['V150834'])
print(inverted)
# inverted = le.inverse_transform([df.])
# print(inverted)

In [ ]:
# OPTIONAL USED FROM CONVERTING EACH VARIETY TO A ONE HOT

def convert_variety_to_one_hot(variety):
    return onehot_encoded[le.transform([variety])]

In [ ]:
# OPTIONAL APPLY ABOVE FUNCTION TO REPLACE EACH ROW IN VARIETY COLUMN WITH ONE HOT REPRESENTATION

df.Variety = df.Variety.apply(lambda val: convert_variety_to_one_hot(val))

# OPTIONAL RESHAPE
df.Variety = df.Variety.apply(lambda val: val.reshape(174))

In [ ]:
# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

print(df)
dummies = pd.get_dummies(df.Variety)
print(dummies)
df = pd.concat([df, dummies], axis=1)
print(df)

In [ ]:
# ORIGINAL COLUMNS! WE WANT TO PICK ONES THAT ARE GOOD FOR OUR LEARNING ALGORITHM AND DROP THE REST IN THE
# FOLLOWING CELL

for col in df.columns:
    print(col, type(df[col][0]))

In [ ]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

df = df.drop(['Experiment', 'Location', 'Planting date', 'Check Yield', 'Yield difference', 'Latitude', 'Longitude', 'Variety', 'PI'], axis=1)


In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [ ]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

# feature_columns = ['Variety', 'Solar Radiation', 'Temperature', 'Precipitation', 'Location', 'Clay', 'Silt', 'Sand', 'pH', 'Precipitation', 'CEC', 'Soil class']

X = df.drop(['Yield'], axis=1)
# X = df.loc[:, feature_columns]

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01, train_size = 0.05, random_state = 42)

train_visual, zz, zzz, zzzz = train_test_split(df, y, test_size=0.01, train_size=0.05, random_state = 42)

from sklearn import preprocessing
from sklearn import utils

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_scalar = StandardScaler()
x_std = standard_scalar.fit_transform(X_train)
print(x_std.shape)



In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0, verbose=1)
x_2d = tsne.fit_transform(x_std)

In [ ]:
print(x_2d.shape)
print(x_2d)
from matplotlib import pyplot as plt
markers=('s', 'd', 'o', '^', 'v')
color_map = {0:'red', 1:'blue', 2:'lightgreen', 3:'purple', 4:'cyan'}
plt.figure()
# for idx, cl in enumerate(np.unique(x_2d)):
#     print(cl)
for idx, cl in enumerate(np.unique(y_train)):
    plt.scatter(x=x_2d[y_train==cl,0], y=x_2d[y_train==cl,1], c=color_map[idx], marker=markers[idx], label=cl)

#     plt.scatter(x=x_2d[cl,0], y=x_2d[cl,1], c=color_map[idx], marker=markers[idx], label=cl)
# plt.show()

In [ ]:
# VISUALIZE FEATURE DISTRIBUTIONS OPTIONALLY

import matplotlib.pyplot as plt
X_train[X_train.dtypes[(X_train.dtypes=="float64")|(X_train.dtypes=="int64")]
                        .index.values].hist(figsize=[11,11])

In [ ]:
# MORE VISUALIZATION OPTIONAL

# Import the class
import kmapper as km

# Some sample data
from sklearn import datasets
# data, labels = datasets.make_circles(n_samples=5000, noise=0.03, factor=0.3)
# print(data)
# print(labels)

# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projected_data = mapper.fit_transform(X_train, projection="knn_distance_5") # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, X_train, nr_cubes=10)

# Visualize it
mapper.visualize(graph, path_html="keplermapper.html",
                 inverse_X_names=['Solar Radiation', 'Temperature', 'Location', 'Precipitation'],
                 title="keplermapper of features")

In [ ]:
# IT COULDN'T HURT TO SEE THE FINAL SHAPES

# print(X_train.describe())
print(X_train.head())
# print(y_train.describe())
print(y_train.head())
print(X_train.shape)
print(y_train.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

errors = np.absolute(((preds - y_test) / y_test) * 100)
print(errors)
print(np.mean(errors))

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE

feature_importances = regr.feature_importances_
feature_importances = pd.Series(feature_importances)
feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
for index, row in feature_importance_df.iterrows():
    print(row['feature'], 'has importance: ', row['feature_importance'])
# for feature_importance in regr.feature_importances_:
    
print(errors.describe())

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
#     linear_model.SGDRegressor(),
#     linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = np.absolute(((preds - y_test) / y_test) * 100)
    print(errors)
    print(np.mean(errors))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import tree
classifier = tree.DecisionTreeClassifier()
classifier.fit(X_train, y_train)